In [ ]:
import sys
import numpy as np
import pickle
from mne_bids import BIDSPath, read_raw_bids
from bids import BIDSLayout
from util.io.coherence import *
from util.io.iter_BIDSPaths import *
from mne_connectivity import spectral_connectivity_time

In [ ]:
SUB = '31'
RUN = '1'
TASK = 'pitch'
FPATH = '/project2/hcn1/pitch_tracking/data/bids/derivatives/preprocessing/sub-31/sub-31_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz'

BIDS_ROOT = '../data/bids'
FIGS_ROOT = '../figs'
DERIV_ROOT = '../data/bids/derivatives'
METHOD = 'coh'
FS = 5000
RAW_TMIN = -0.2
RAW_TMAX = 0.5
TMIN = 0
TMAX = 0.25
N_CHANS = 62
CONDS = ['50', '100', '150', '200', '250']
FREQS = [50, 100, 150, 200, 250]

In [4]:
# Load epoched data
epochs = mne.read_epochs(FPATH, preload = True)
events = epochs.events
n_epochs = len(events)

Reading /project2/hcn1/pitch_tracking/data/bids/derivatives/preprocessing/sub-31/sub-31_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Reading /project2/hcn1/pitch_tracking/data/bids/derivatives/preprocessing/sub-31/sub-31_task-pitch_run-1_res-hi_desc-clean_epo.fif-1.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
0 bad epochs dropped
Not setting metadata
4801 matching events found
No baseline correction applied
0 projection items activated


In [5]:
# Use a different sub for generating stim channels if sub has bad Aux channel
STIM_SUB, STIM_RUN = get_stim_sub(SUB, RUN)

In [6]:
# Create epochs from raw data to create simulated stim channels
raw_epochs = get_raw_epochs(BIDS_ROOT, STIM_SUB, TASK, STIM_RUN)
stim_epochs_array = create_stim_epochs_array(raw_epochs, n_epochs, CONDS)
simulated_epochs = create_stim_epochs_object(stim_epochs_array, events, CONDS, FS, RAW_TMIN)

Used Annotations descriptions: ['100', '150', '200', '250', '50']
Not setting metadata
6000 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1180 events and 3501 original time points ...


/project2/hcn1/pitch_tracking/scripts/util/io/coherence.py:42: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['leog', 'reog', 'Aux1']
  raw = read_raw_bids(bids_path, verbose = False)
/project2/hcn1/pitch_tracking/scripts/util/io/coherence.py:42: RuntimeWarning: Not setting position of 1 stim channel found in montage:
['Aux1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = read_raw_bids(bids_path, verbose = False)


0 bad epochs dropped
Loading data for 1234 events and 3501 original time points ...
0 bad epochs dropped
Loading data for 1216 events and 3501 original time points ...
0 bad epochs dropped
Loading data for 1219 events and 3501 original time points ...
0 bad epochs dropped
Loading data for 1151 events and 3501 original time points ...
0 bad epochs dropped
Not setting metadata
4801 matching events found
Applying baseline correction (mode: mean)
0 projection items activated


/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/mne/io/meas_info.py:864: UserWarning: Avoid manually setting custom_ref_applied cannot be set directly. Please use method inst.set_eeg_reference() instead.
  warnings.warn(f"Avoid manually setting {self._attributes[key]}")
/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/mne/io/meas_info.py:864: UserWarning: Avoid manually setting highpass cannot be set directly. Please use method inst.filter() instead.
  warnings.warn(f"Avoid manually setting {self._attributes[key]}")
/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/mne/io/meas_info.py:864: UserWarning: Avoid manually setting lowpass cannot be set directly. Please use method inst.filter() instead.
  warnings.warn(f"Avoid manually setting {self._attributes[key]}")


0 bad epochs dropped


In [7]:
# Crop data so both epoch objects have same windowing
simulated_epochs = simulated_epochs.crop(tmin = TMIN, tmax = TMAX)
epochs = epochs.crop(tmin = TMIN)

In [8]:
# Add simulated channels to data
combined_epochs = mne.epochs.add_channels_epochs([epochs, simulated_epochs])

In [30]:
# n_epochs = 984
# for i in range(100, n_epochs, 100):
#     print(i)
# n_epochs_list = list(range(100, n_epochs, 100))
# n_epochs_list.append(n_epochs)
# n_epochs_list
epoch_indexes = list(range(10))
epoch_indexes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
# Compute coherence
fmin, fmax = get_fmin_and_fmax(CONDS)
coh_df = pd.DataFrame()
indices = get_coh_indices(N_CHANS)

# Compute coherence over each condition
for cond in CONDS: 
    cond_epochs = combined_epochs[cond]
    
    # Index over 100 epochs at a time
    start_indexes = list(range(100, n_epochs, 100))
    start_indexes.insert(0, 0)
    end_indexes = list(range(100, n_epochs, 100))
    n_epochs = cond_epochs.__len__()
    end_indexes.append(n_epochs)
    
    for start, end in zip(start_indexes, end_indexes):
        print(f'start: {start}, end: {end}')
        
        # Compute coherence over 100 epochs
        cond_epochs_slice = cond_epochs[start:end]
        coh = spectral_connectivity_time(
            cond_epochs_slice,
            freqs = FREQS,
            method = 'coh', 
            sfreq = FS, 
            n_jobs = 1)
        break
        
    # save_file_name = 'subj-' + SUB + '_task-pitch_cond-' + cond + '_coh_by_epoch.pkl'
    # save_dir = '/project2/hcn1/pitch_tracking/data/bids/derivatives/coherence'
    # save_path = save_file_name + save_dir
    # with open(save_path, 'wb') as file:
    #     pickle.dump(std1, save_path)
    #     print(f'Object successfully saved to "{save_path}"')
    break



start: 0, end: 100
Adding metadata with 3 columns
Connectivity computation...


  0%|          |  : 0/2278 [00:00<?,       ?it/s]

In [24]:
cond_epochs_slice

Number of events,100
Events,50: 100
Time range,0.000 – 0.250 sec
Baseline,-0.200 – 0.000 sec


In [12]:
cond_epochs = combined_epochs[cond]
n_epochs = cond_epochs.__len__()
n_epochs_list = list(range(100, n_epochs, 100))
n_epochs_list.append(n_epochs)

In [15]:
n_epochs_list

[100, 200, 300, 400, 500, 600, 700, 800, 900, 956]